In [1]:
param_date = '20240623'

In [2]:
import pandas as pd

output_correct, output_error = pd.DataFrame(), pd.DataFrame()

In [3]:
input_ = pd.read_excel(f'./{param_date}/part.xlsx',
                       header=0,
                       dtype=str).fillna('')

input_ = input_.drop_duplicates(['JOIN_PTPKPNCN'], keep='first', ignore_index=True)

print(f'总数量：{len(input_)}')
input_['JOIN_PTPKPNCN'].tolist()

总数量：24720


['131;10671829;BLACKHAT;ROCKAUTO',
 '131;15108561;B2;ROCKAUTO',
 '131;15108557;B1;ROCKAUTO',
 '131;15108565;B3;ROCKAUTO',
 '1002098;15104205;H1;ROCKAUTO',
 '1002098;15104209;H2;ROCKAUTO',
 '256;9097025;T1;ROCKAUTO',
 '256;10512261;T2;ROCKAUTO',
 '256;14919165;T3;ROCKAUTO',
 '256;15138777;T10;ROCKAUTO',
 '256;15138781;T11;ROCKAUTO',
 '256;15138773;T9;ROCKAUTO',
 '256;15521065;T3C;ROCKAUTO',
 '256;14922125;T4;ROCKAUTO',
 '256;14922129;T5;ROCKAUTO',
 '256;15001073;T6;ROCKAUTO',
 '256;15138765;T7;ROCKAUTO',
 '256;15138769;T8;ROCKAUTO',
 '256;15521069;T3R;ROCKAUTO',
 '260;1285405;GIFTCERTVAR;ROCKAUTO',
 '260;368849;GIFTCERT25;ROCKAUTO',
 '260;368850;GIFTCERT50;ROCKAUTO',
 '260;368851;GIFTCERT75;ROCKAUTO',
 '260;368852;GIFTCERT100;ROCKAUTO',
 '1001191;2961354;LANYARD;ROCKAUTO',
 '1000000;1272469;MAGPACK;ROCKAUTO',
 '1001187;2867484;BUMPERDECAL;ROCKAUTO',
 '1001187;3472857;BUMPERDECAL2;ROCKAUTO',
 '1001187;2867487;WINDOWDECAL;ROCKAUTO',
 '1001187;2867490;DECALCOMBO;ROCKAUTO',
 '1001187;360357

In [4]:
crawler_status = 'error'

crawler_status

'error'

In [5]:
import requests

import sys
sys.path.append('../../00.Tools')
import UA
import Proxy

In [6]:
a = 0

data = {'func': 'getbuyersguide',
        'payload': f'''{{"partData":{{"listing_data_essential":{{"parttype":"{input_.loc[a, 'Part PartType']}","partkey":"{input_.loc[a, 'PartKey']}"}},"listing_data_supplemental":{{"partnumber":"{input_.loc[a, 'PartNumber']}","catalogname":"{input_.loc[a, 'CatalogName']}"}}}}}}''',
        'partsearch[manufacturer][partsearch_007]': input_.loc[a, 'CatalogName'],
        'api_json_request': '1'}

resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                     data=data,
                     headers=UA.get_User_Agent_Requests(),
                     proxies=Proxy.get_Proxy_Requests()).text

resp

'{"buyersguidepieces":{"body":"No applications found.","title":"Buyer\'s Guide : ROCKAUTO BLACKHAT Hat"},"groupindex":null,"collected_javascript":"\\/**UQ:80EDFADE**\\/\\nwindow.top.parent.window[\\"_nck\\"] = \\"8LVLUxTaILBT+tLxkg8+3Ag7\\\\\\/0bbOhvn7P3rkFbL80\\\\\\/ismhHyfCiO07pwNUCaIW3afbJMKR0uqi1pNnUlRgBN5CwkZnWFYwm7DtiVw8cl6t1DDIlw0OPK8L1xqUF2GhWh7smW3GvoaM18\\\\\\/NQ7ls3Rfzk2YGUOYLvham585c0CDU=\\";\\n"}'

In [7]:
import json

In [8]:
json_ = json.loads(resp)

json_

{'buyersguidepieces': {'body': 'No applications found.',
  'title': "Buyer's Guide : ROCKAUTO BLACKHAT Hat"},
 'groupindex': None,
 'collected_javascript': '/**UQ:80EDFADE**/\nwindow.top.parent.window["_nck"] = "8LVLUxTaILBT+tLxkg8+3Ag7\\/0bbOhvn7P3rkFbL80\\/ismhHyfCiO07pwNUCaIW3afbJMKR0uqi1pNnUlRgBN5CwkZnWFYwm7DtiVw8cl6t1DDIlw0OPK8L1xqUF2GhWh7smW3GvoaM18\\/NQ7ls3Rfzk2YGUOYLvham585c0CDU=";\n'}

In [9]:
from bs4 import BeautifulSoup
from lxml import etree

In [10]:
soup = BeautifulSoup(json_['buyersguidepieces']['body'], 'lxml')
html = etree.HTML(str(soup))

with open('./vehicle.html', 'w', encoding='UTF-8') as file:
    file.write(str(soup))

print(soup.prettify())

<html>
 <body>
  <p>
   No applications found.
  </p>
 </body>
</html>



In [11]:
list_tr = html.xpath('//tr')

list_tr

[]

In [12]:
dict_vehicle = {}
for i in range(len(list_tr)):
    dict_vehicle[str(i)] = {}
    list_td = [td.strip() for td in list_tr[i].xpath('./td/text()')]
    for j in range(len(list_td)):
        dict_vehicle[str(i)][str(j)] = list_td[j]

json_vehicle = json.dumps(dict_vehicle)

json_vehicle

'{}'

In [13]:
vehicle = 'No applications found.' if json_['buyersguidepieces']['body'].strip() == 'No applications found.' else ''
for i in dict_vehicle:
    for j in dict_vehicle[i]:
        vehicle += dict_vehicle[i][j] + ' '
    vehicle = vehicle.strip() + '\n'

vehicle = vehicle.strip()

print(vehicle)

No applications found.


In [14]:
df_temp = pd.DataFrame([{'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                         'Json_Vehicle': json_vehicle,
                         'Vehicle': vehicle}])

output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')

output_correct

,JOIN_PTPKPNCN,Json_Vehicle,Vehicle
0,131;10671829;BLACKHAT;ROCKAUTO,{},No applications found.


In [15]:
crawler_status = 'ok'

crawler_status

'ok'

In [16]:
if crawler_status == 'error':
    df_temp = pd.DataFrame([{'PartType': input_.loc[a, 'PartType'],
                             'Tab Url': input_.loc[a, 'Tab Url'],
                             'CatalogName': input_.loc[a, 'CatalogName'],
                             'PartNumber': input_.loc[a, 'PartNumber'],
                             'Part PartType': input_.loc[a, 'Part PartType'],
                             'PartKey': input_.loc[a, 'PartKey'],
                             'JOIN_PTPKPNCN': input_.loc[a, 'JOIN_PTPKPNCN'],
                             'Url': input_.loc[a, 'Url']}])

    output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

output_error

""


In [17]:
if len(output_correct) != 0:
    output_correct = output_correct.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_correct.to_excel('./test_vehicle.xlsx', index=False)

output_correct

,JOIN_PTPKPNCN,Json_Vehicle,Vehicle
0,131;10671829;BLACKHAT;ROCKAUTO,{},No applications found.


In [18]:
if len(output_error) != 0:
    output_error = output_error.sort_values(by=['JOIN_PTPKPNCN'], ascending=[True], ignore_index=True)
    output_error.to_excel('./test_vehicle_error.xlsx', index=False)

output_error

""
